### Gradient Explorations

This document includes the explorations of non-linear boundary avoidance. It is missing some code that I have locally and will put up Soon (TM).

In [ ]:
%load_ext autoreload
%autoreload 2


import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from scipy.optimize import minimize, LinearConstraint
from scipy.interpolate import interp1d, LinearNDInterpolator

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
os.chdir('../src/')


# Import whatever we need
import disruptivity as dis
from boundary_avoidance import *
import vis.disruptivity_vis as dis_vis
import vis.probability_vis as prob_vis
from vis.plot_helpers import plot_subplot as plot
from data_loader import load_disruptions_mat as load_disruptions_mat

# Import tokamak Configuartions
import tokamaks.cmod as cmod
CMOD = cmod.CONFIG

# C-Mod

Working with disruptivity always starts with loading the data.

In [ ]:
# Loading
dataframe, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')
cmod_df = dataframe

Create an example entry dictionary.

In [ ]:
# Entry dictionary
entry_dict = {
    'z_error': CMOD["entry_dict"]["z_error"],
    'kappa': CMOD["entry_dict"]["kappa"],
}

In [ ]:
# Computing the disruptivity and plotting the regular figure
# generate a shotlist that does not have the UFOs
ufo_shotlist = np.loadtxt("../data/cmod_ufo_shotlist.txt")
all_disrupted_pulses = np.array(cmod_df.shot[cmod_indices["indices_disrupt_time_in_flattop"]])
shot_indices = np.isin(all_disrupted_pulses, ufo_shotlist).astype(bool)
shotlist_no_ufo = all_disrupted_pulses[np.logical_not(shot_indices)]
cmod_vde_shotlist = np.loadtxt("../data/cmod_vde_shotlist.txt", dtype=int)

# Compute the disruptivity
shotlist=shotlist_no_ufo
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, dataframe, cmod_indices, shotlist=shotlist, tau=50, window=25)
args = dis.compute_disruptivity_sampling(dataframe, entry_dict, indices_n_disrupt, indices_n_total, nbins=35)

In [ ]:
# Different pulses for overlay.
shot = 1140226013 #AT PULSE
# shot = 1160930030 # BOB PULSE
# shot = 1120105021 #VDE
# shot = 1120831007 # Non disrupted

fig,ax = plot('cmod_q95_ne_disruptivity_kaloyannis.png', dis_vis.subplot_disruptivity2d, args, figsize=(7,4))
ax.set_title(f"Shot {shot}")
dis_vis.subplot_draw_trajectory(
    ax, 
    dataframe, 
    entry_dict, 
    cmod_indices['indices_flattop'], 
    shot
)
plt.savefig("nofill_trajectory.png", dpi=400, facecolor='white', bbox_inches="tight")

In [ ]:
# Now interatively fill the grid
iter_array = iterative_fill(args[0], args[2])

# Prep the interpolator for the value of the disruptivity
bin_centers = (np.array(args[2])[:,1:]+np.array(args[2])[:,:-1])/2
xx = np.meshgrid(*bin_centers)
interper = scipy.interpolate.RegularGridInterpolator(bin_centers, iter_array,
                                                     method='linear',
                                                     bounds_error=False,
                                                     fill_value=iter_array.min())

In [ ]:
# Visualize the filling
args2 = list(args)
args2[0]=iter_array
fig,ax = plot('iter_fill.png', dis_vis.subplot_disruptivity2d, args2, figsize=(7,4))
ax.set_title(f"Shot {shot}")
dis_vis.subplot_draw_trajectory(
    ax, 
    dataframe, 
    entry_dict, 
    cmod_indices['indices_flattop'], 
    shot
)
plt.savefig("nograd_trajectory.png", dpi=400, facecolor='white', bbox_inches="tight")

In [ ]:
# Now Compute the Gradients
fig,ax = plt.subplots(1,3, figsize = (15,4) , constrained_layout=True)

#Gradients
bins = args2[2]
bin_centers = (np.array(bins)[:,1:]+np.array(bins)[:,:-1])/2
grad_x, grad_y = compute_grads(args2[0], args2[2])
norm = np.sqrt(grad_x**2+grad_y**2)
interper_x, interper_y = create_interpers(grad_x, grad_y, bins)

dis_vis.subplot_grad2d(ax[0], grad_x, *args[1:])
dis_vis.subplot_grad2d(ax[1], grad_y, *args[1:])
dis_vis.subplot_grad2d(ax[2], norm, *args[1:])

ax[0].set_title("Grad X")
ax[1].set_title("Grad Y")
ax[2].set_title("Grad Norm")